In [18]:
import pandas as pd
import numpy as np

In [19]:
listings_data = pd.read_csv('listings.csv.gz')

In [20]:
listings_data

,id,listing_url,scrape_id,last_scraped,name,description,neighborhood_overview,picture_url,host_id,host_url,...,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,10080,https://www.airbnb.com/rooms/10080,20210412154228,2021-04-13,D1 - Million Dollar View 2 BR,"Stunning two bedroom, two bathroom apartment. ...",NaN,https://a0.muscache.com/pictures/55778229/c2f7...,30899,https://www.airbnb.com/users/show/30899,...,9.0,10.0,9.0,18-476608,f,40,40,0,0,0.14
1,13358,https://www.airbnb.com/rooms/13358,20210412154228,2021-04-13,Monthly (or Longer ) Designer One Bedroom Down...,<b>The space</b><br />This suites central loca...,NaN,https://a0.muscache.com/pictures/c23bb7ef-e300...,52116,https://www.airbnb.com/users/show/52116,...,10.0,10.0,9.0,18-611603,f,1,1,0,0,3.27
2,13490,https://www.airbnb.com/rooms/13490,20210412154228,2021-04-13,Vancouver's best kept secret,This apartment rents for one month blocks of t...,"In the heart of Vancouver, this apartment has ...",https://a0.muscache.com/pictures/73394727/79d5...,52467,https://www.airbnb.com/users/show/52467,...,10.0,10.0,10.0,NaN,f,1,1,0,0,0.73
3,14267,https://www.airbnb.com/rooms/14267,20210412154228,2021-04-13,EcoLoft Vancouver,"The Ecoloft is located in the lovely, family r...",We live in the centre of the city of Vancouver...,https://a0.muscache.com/pictures/3646de9b-934e...,56030,https://www.airbnb.com/users/show/56030,...,9.0,10.0,9.0,21-156500,t,1,1,0,0,0.26
4,16254,https://www.airbnb.com/rooms/16254,20210412154228,2021-04-12,Close to PNE/Hastings Park and East Village,2 Bedroom garden level guest suite.(lower leve...,"Good Eats, Cute Shops and all the personality ...",https://a0.muscache.com/pictures/90623667/583c...,63238,https://www.airbnb.com/users/show/63238,...,10.0,10.0,9.0,19-162421,t,1,1,0,0,0.15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4294,49132295,https://www.airbnb.com/rooms/49132295,20210412154228,2021-04-13,Luxurious 3 Bedroom Unit With English Bay Views!,"Stunning large, modern, brand-new 3 bedroom an...",NaN,https://a0.muscache.com/pictures/d5f23f5b-f10b...,219146817,https://www.airbnb.com/users/show/219146817,...,NaN,NaN,NaN,NaN,f,5,5,0,0,NaN
4295,49138797,https://www.airbnb.com/rooms/49138797,20210412154228,2021-04-13,19-9-10 Private room,Private room in a new house with shared bathro...,NaN,https://a0.muscache.com/pictures/02048308-7b1f...,356993612,https://www.airbnb.com/users/show/356993612,...,NaN,NaN,NaN,21-107408,f,14,1,13,0,NaN
4296,49146266,https://www.airbnb.com/rooms/49146266,20210412154228,2021-04-13,Prime Coal harbour | Luxury Finishings,Welcome to this beautiful fully furnished Prim...,"Location, Location, Location! Steps from the S...",https://a0.muscache.com/pictures/ee697899-6185...,396405989,https://www.airbnb.com/users/show/396405989,...,NaN,NaN,NaN,NaN,f,1,1,0,0,NaN
4297,49148812,https://www.airbnb.com/rooms/49148812,20210412154228,2021-04-13,DT- Ocean View 1BD w/ Private Bathroom Kitchen...,Private one bedroom suite apartment in heart o...,"Heart of Downtown Vancouver , walking distance...",https://a0.muscache.com/pictures/02a2a9da-68e0...,88405999,https://www.airbnb.com/users/show/88405999,...,NaN,NaN,NaN,NaN,f,1,1,0,0,NaN


In [21]:
listings_data.columns

Index(['id', 'listing_url', 'scrape_id', 'last_scraped', 'name', 'description',
       'neighborhood_overview', 'picture_url', 'host_id', 'host_url',
       'host_name', 'host_since', 'host_location', 'host_about',
       'host_response_time', 'host_response_rate', 'host_acceptance_rate',
       'host_is_superhost', 'host_thumbnail_url', 'host_picture_url',
       'host_neighbourhood', 'host_listings_count',
       'host_total_listings_count', 'host_verifications',
       'host_has_profile_pic', 'host_identity_verified', 'neighbourhood',
       'neighbourhood_cleansed', 'neighbourhood_group_cleansed', 'latitude',
       'longitude', 'property_type', 'room_type', 'accommodates', 'bathrooms',
       'bathrooms_text', 'bedrooms', 'beds', 'amenities', 'price',
       'minimum_nights', 'maximum_nights', 'minimum_minimum_nights',
       'maximum_minimum_nights', 'minimum_maximum_nights',
       'maximum_maximum_nights', 'minimum_nights_avg_ntm',
       'maximum_nights_avg_ntm', 'calendar_upd

In [22]:
col = ['last_review', 'review_scores_rating', 'review_scores_accuracy',
       'review_scores_cleanliness', 'review_scores_checkin',
       'review_scores_communication', 'review_scores_location',
       'review_scores_value']

In [23]:
lis = listings_data[col]

In [24]:
lis

,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value
0,2017-02-26,93.0,9.0,9.0,10.0,9.0,10.0,9.0
1,2021-01-01,93.0,9.0,10.0,9.0,10.0,10.0,9.0
2,2020-10-31,98.0,10.0,10.0,10.0,10.0,10.0,10.0
3,2020-02-07,95.0,10.0,9.0,10.0,9.0,10.0,9.0
4,2019-08-19,96.0,10.0,10.0,10.0,10.0,10.0,9.0
...,...,...,...,...,...,...,...,...
4294,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4295,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4296,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4297,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
lis["review_scores_rating"].isna().sum()

825

In [5]:
columns_needed = ['id', 'listing_url', 'name', 'description', 'picture_url', 'latitude', 'longitude', 'property_type', 'accommodates', 'bedrooms', 'beds', 'amenities', 'price' ]

In [6]:
listings_data = listings_data[columns_needed]

In [7]:
listings_data

,id,listing_url,name,description,picture_url,latitude,longitude,property_type,room_type,accommodates,bedrooms,beds,amenities,price,review_scores_rating,review_scores_value
0,10080,https://www.airbnb.com/rooms/10080,D1 - Million Dollar View 2 BR,"Stunning two bedroom, two bathroom apartment. ...",https://a0.muscache.com/pictures/55778229/c2f7...,49.28872,-123.12046,Entire condominium,Entire home/apt,5,2.0,2.0,"[""Iron"", ""Dryer"", ""Bed linens"", ""Cooking basic...",$150.00,93.0,9.0
1,13358,https://www.airbnb.com/rooms/13358,Monthly (or Longer ) Designer One Bedroom Down...,<b>The space</b><br />This suites central loca...,https://a0.muscache.com/pictures/c23bb7ef-e300...,49.28201,-123.12669,Entire condominium,Entire home/apt,2,1.0,1.0,"[""Iron"", ""Dryer"", ""Bed linens"", ""Cooking basic...",$85.00,93.0,9.0
2,13490,https://www.airbnb.com/rooms/13490,Vancouver's best kept secret,This apartment rents for one month blocks of t...,https://a0.muscache.com/pictures/73394727/79d5...,49.25622,-123.06607,Entire apartment,Entire home/apt,2,1.0,1.0,"[""Iron"", ""Outdoor dining area"", ""Ethernet conn...",$145.00,98.0,10.0
3,14267,https://www.airbnb.com/rooms/14267,EcoLoft Vancouver,"The Ecoloft is located in the lovely, family r...",https://a0.muscache.com/pictures/3646de9b-934e...,49.24922,-123.08139,Entire house,Entire home/apt,4,1.0,2.0,"[""Iron"", ""Dryer"", ""Cooking basics"", ""Hot water...",$140.00,95.0,9.0
4,16254,https://www.airbnb.com/rooms/16254,Close to PNE/Hastings Park and East Village,2 Bedroom garden level guest suite.(lower leve...,https://a0.muscache.com/pictures/90623667/583c...,49.27721,-123.04086,Entire guest suite,Entire home/apt,4,2.0,3.0,"[""Iron"", ""Dryer"", ""Bed linens"", ""Cooking basic...",$195.00,96.0,9.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4294,49132295,https://www.airbnb.com/rooms/49132295,Luxurious 3 Bedroom Unit With English Bay Views!,"Stunning large, modern, brand-new 3 bedroom an...",https://a0.muscache.com/pictures/d5f23f5b-f10b...,49.27391,-123.13012,Entire apartment,Entire home/apt,7,3.0,5.0,"[""Heating"", ""Essentials"", ""Kitchen"", ""Iron"", ""...",$500.00,NaN,NaN
4295,49138797,https://www.airbnb.com/rooms/49138797,19-9-10 Private room,Private room in a new house with shared bathro...,https://a0.muscache.com/pictures/02048308-7b1f...,49.25275,-123.09184,Private room in house,Private room,2,1.0,1.0,"[""Heating"", ""Kitchen"", ""Fire extinguisher"", ""D...",$38.00,NaN,NaN
4296,49146266,https://www.airbnb.com/rooms/49146266,Prime Coal harbour | Luxury Finishings,Welcome to this beautiful fully furnished Prim...,https://a0.muscache.com/pictures/ee697899-6185...,49.28631,-123.12455,Entire apartment,Entire home/apt,2,1.0,NaN,"[""Kitchen"", ""Body soap"", ""Shampoo"", ""Hot tub"",...",$160.00,NaN,NaN
4297,49148812,https://www.airbnb.com/rooms/49148812,DT- Ocean View 1BD w/ Private Bathroom Kitchen...,Private one bedroom suite apartment in heart o...,https://a0.muscache.com/pictures/02a2a9da-68e0...,49.28822,-123.13061,Entire apartment,Entire home/apt,1,1.0,1.0,"[""Dryer"", ""Bed linens"", ""Elevator"", ""Paid park...",$66.00,NaN,NaN
